# Конвертация данных из сервиса [DOTA](https://captain-whu.github.io/DOTA/) в формат для обусения модели YOLO

In [ ]:
import os
from PIL import Image

def convert_dota_to_yolo(dota_annotation_path, image_path, output_path):
    # Читаем DOTA аннотацию
    with open(dota_annotation_path, 'r') as f:
        lines = f.readlines()
    
    # Пропускаем первые две строки (imagesource и gsd)
    annotations = lines[2:]
    
    # Получаем размеры изображения
    with Image.open(image_path) as img:
        img_width, img_height = img.size
    
    # Преобразуем каждую аннотацию
    yolo_annotations = []
    for ann in annotations:
        parts = ann.strip().split()
        if len(parts) < 9:
            continue
            
        # Координаты полигона (8 значений)
        poly = list(map(float, parts[:8]))
        class_name = parts[8]
        
        # Преобразуем полигон в bounding box
        x_coords = poly[0::2]
        y_coords = poly[1::2]
        x_min, x_max = min(x_coords), max(x_coords)
        y_min, y_max = min(y_coords), max(y_coords)
        
        # Конвертируем в YOLO формат (нормализованные координаты)
        center_x = (x_min + x_max) / 2 / img_width
        center_y = (y_min + y_max) / 2 / img_height
        width = (x_max - x_min) / img_width
        height = (y_max - y_min) / img_height
        
        # Определяем class_id (у нас пока только small-vehicle)
        class_id = 0  # small-vehicle
        
        yolo_annotations.append(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}")
    
    # Сохраняем YOLO аннотации
    with open(output_path, 'w') as f:
        f.write('\n'.join(yolo_annotations))

# Преобразуем все аннотации
for split in ['train']:
    image_dir = f"dataset/images/{split}"
    label_dir = f"dataset/labels/{split}"
    
    for img_file in os.listdir(image_dir):
        if img_file.endswith(('.jpg', '.png', '.jpeg')):
            base_name = os.path.splitext(img_file)[0]
            dota_annotation_path = f"{label_dir}/{base_name}.txt"
            image_path = f"{image_dir}/{img_file}"
            yolo_output_path = f"{label_dir}/{base_name}.txt"  # перезаписываем
            
            if os.path.exists(dota_annotation_path):
                convert_dota_to_yolo(dota_annotation_path, image_path, yolo_output_path)
                print(f"Converted {dota_annotation_path}")

Converted dataset/labels/train/P0000.txt
Converted dataset/labels/train/P0001.txt
Converted dataset/labels/train/P0002.txt
Converted dataset/labels/train/P0005.txt
Converted dataset/labels/train/P0008.txt
Converted dataset/labels/train/P0010.txt
Converted dataset/labels/train/P0011.txt


# Обучение нашей елочки

In [26]:
from ultralytics import YOLO
import torch

# Загружаем предобученную модель
model = YOLO("yolo11n.pt")

results = model.train(
    data='data.yml',
    epochs=1,
    batch=2,    
    imgsz=640,
    project='C:/Users/Smart/PycharmProjects/Data_analysis/Drones',
    device='cpu',
    name='small_dataset_test',
    patience=10,
    save=True,
    augment=True,  # Включим аугментацию
    degrees=10.0,
    translate=0.1,
    scale=0.2,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    lr0=0.01,
    lrf=0.1
)

Ultralytics 8.3.200  Python-3.11.5 torch-2.8.0+cpu CPU (11th Gen Intel Core i5-1135G7 @ 2.40GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=2, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yml, degrees=10.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.1, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=small_dataset_test, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plots=True, pose

# Тест модельки

In [28]:
from ultralytics import YOLO
import cv2
import os

# Загружаем обученную модель
model = YOLO('C:\\Users\\Smart\\PycharmProjects\\Data_analysis\\Drones\\small_dataset_test\\weights\\best.pt')

# Тестируем на одном изображении из тренировочного набора
test_image_path = "C:\\Users\\Smart\\PycharmProjects\\Data_analysis\\Drones\\dataset\\images\\train\\P0000.png"

# Делаем предсказание
results = model(test_image_path)

# Визуализируем результат
for result in results:
    # Рисуем bounding boxes
    result.show()
    result.save("prediction_result.jpg")
    
    # Выводим информацию об обнаруженных объектах
    for box in result.boxes:
        print(f"Обнаружен объект: confidence={box.conf[0]:.2f}, class={model.names[int(box.cls[0])]}")


image 1/1 C:\Users\Smart\PycharmProjects\Data_analysis\Drones\dataset\images\train\P0000.png: 640x480 (no detections), 672.8ms
Speed: 27.9ms preprocess, 672.8ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)
